In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
import requests
import networkx as nx
import urllib.parse
from math import radians, sin, cos, atan2, sqrt
from pprint import pprint

In [2]:
df = pd.read_csv('1854 WCR 2023.csv',header=1)
df['Where Bound'] = df['Where Bound'].apply(lambda x: str(x).strip())
df['Where From'] = df['Where From'].apply(lambda x: str(x).strip())
exclude_list = ['?','','nan']
df = df[(~df['Where Bound'].isin(exclude_list)) & (~df['Where From'].isin(exclude_list))]

In [3]:
location_strings = set(df['Where Bound'])
location_strings.update(df['Where From'])
location_strings = sorted(location_strings)

In [4]:
with open('water_paths.json') as wp_geojson:
    gj = json.loads(wp_geojson.read())
    

In [5]:
def feature_filter(gj, gid_list=None, exclude=True):
    '''filter out features from geojson. optionally exclude or include only features with gid in gid_list.'''
    filtered = []
    for f in gj['features']:
        if not gid_list:
            filtered.append(f)
            continue
            
        if exclude and f['properties']['gid'] not in gid_list: 
            filtered.append(f)
        elif not exclude and f['properties']['gid'] in gid_list: 
            filtered.append(f)
    
    return filtered

def distance(p1, p2):
    '''returns distance in KM between two points'''
    earth_radius = 6371
    lat1, lat2 = radians(p1[1]), radians(p2[1])
    lon1, lon2 = radians(p1[0]), radians(p2[0])
    
    #https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    base = earth_radius * c
    return base

def draw_graph(G):
    pos = nx.planar_layout(G)
    nx.draw(G, pos, with_labels=True, font_weight='bold')
    el=nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G,pos,edge_labels=el)

def path_distance(g, path):
    prev = path[0]
    distances = []
    for p in path[1:-1]:
        distances.append(g.edges[(prev,p)]['weight'])
        prev = p
    p = path[-1]
    distances.append(g.edges[(prev,p)]['weight'])
    return sum(distances)

def geojson_to_graph(features):
    G = nx.Graph()
    for feat in features:
        feat_geo = feat['geometry']['coordinates']
        anode = str(feat['properties']['anode'])
        bnode = str(feat['properties']['bnode'])
        #length = feat['properties']['length']
        id = feat['properties']['OBJECTID']
        id_postfix = 1
        G.add_node(anode, coordinates=tuple(feat_geo[0]))
        G.add_node(bnode, coordinates=tuple(feat_geo[-1]))

        prev = anode
        prev_coord = feat_geo[0]
        for point in feat_geo[1:-1]:
            name = f"{id}_{id_postfix}"
            id_postfix += 1
            G.add_node(name, coordinates=tuple(point))
            G.add_edge(prev,name, weight=distance(prev_coord, point))
            prev = name
            prev_coord = point
        G.add_edge(prev, bnode, weight=distance(prev_coord, feat_geo[-1]))
    return G

#Exclude the lone multiline path in the geojson data
#this is okay since it its a series of disconnectd strings anyways
features = feature_filter(gj, [837], exclude=True)
g = geojson_to_graph(features)


In [6]:
def coordinates_set(g):
    return {g.nodes[node]['coordinates'] for node in g.nodes}

def closest_coordinate(all_cords, candidate_points):
    point_distances = []
    for point in candidate_points:
        #find the closest node coordinate in the path graph to this point
        cc = sorted(all_cords, key=lambda coord: distance(coord, point))[0]
        #note down the point and the distance to its closest coordinate
        point_distances.append((cc, point, distance(cc, point)))
    
    #return the pair that has the smallest distance
    return sorted(point_distances, key=lambda record: record[2])[0]

def get_features_coords(features):
    return [feat['center'] for feat in features]

def location_name_to_coord(g, locations):
    '''Using the geocoding api, map locations to their most likely lat/lon'''
    req_url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?proximity=ip&access_token=pk.eyJ1IjoiZmpvaG5zODg4IiwiYSI6ImNsaGh6dnh2ajAzNDkzcXM1OWxwOWF1amIifQ._g2Uwo_6wLTw12W5R-b57w'
    mapping = {}
    all_coords = coordinates_set(g)
    
    for loc in locations:
        features = requests.get(req_url.format(urllib.parse.quote(loc))).json()
        assert len(features['features']), features
        candidate_points = get_features_coords(features['features'])
        cc = closest_coordinate(all_coords, candidate_points)
        mapping[loc] = {'graphcoord':cc[0], 'geocoded_coord':cc[1]}
    
    return mapping

mapping = location_name_to_coord(g,location_strings)




In [12]:
def mapping_to_geojson(mapping):
    '''Dump location lon/dat data'''
    
    features = {
      "type": "FeatureCollection",
      "features": []
    }
    for loc in mapping:
        loc_gj = {"type": "Feature",
                  "id": loc,
                  "geometry": {
                      "type": "Point", 
                      "coordinates": mapping[loc]['graphcoord']},
                  "properties": {"name": loc}}
        features['features'].append(loc_gj)
    return json.dumps(features)

def mapping_to_geojson_debug(mapping):
    '''Dump queried/looked up lon/lat locations with their most likely graph locations.
    Provided as a LineString so that the discrepancy can be observed'''
    
    linefeatures = {
      "type": "FeatureCollection",
      "features": []
    }
    estfeatures = {
      "type": "FeatureCollection",
      "features": []
    }
    realfeatures = {
      "type": "FeatureCollection",
      "features": []
    }
        
    for loc in mapping:
        loc_gj = {"type": "Feature",
                  "id": loc,
                  "geometry": {
                      "type": "LineString", 
                      "coordinates": [mapping[loc]['graphcoord'], mapping[loc]['geocoded_coord']]},
                  "properties": {"name": loc}}
        est = {"type": "Feature",
               "id": loc,
               "geometry":{"type":"Point", "coordinates":mapping[loc]['graphcoord']},
               "properties":{"name":loc}}
        real = {"type": "Feature",
               "id": loc,
               "geometry":{"type":"Point", "coordinates":mapping[loc]['geocoded_coord']},
               "properties":{"name":loc}}
        linefeatures['features'].append(loc_gj) 
        estfeatures['features'].append(src)
        realfeatures['features'].append(dst)
    return list(map(json.dumps, [linefeatures, estfeatures, realfeatures]))


In [16]:
# with open('locmap.json','w') as lf:
#     lf.write(json.dumps(mapping))
# with open('bases.json','w') as bf:
#     bf.write(mapping_to_geojson(mapping))

line, src, dst = mapping_to_geojson_debug(mapping)
with open('bases_lines.json','w') as bf:
     bf.write(line)
with open('bases_est.json','w') as bf:
    bf.write(src)
with open('bases_real.json','w') as bf:
    bf.write(dst)

In [61]:
def path_to_coordinates(g, path):
    return [g.nodes[node]['coordinates'] for node in path]

def coordinates_to_geojson(coordinates):
    geo_json = {"type": "Feature",
                "properties":{},
                "geometry":{
                    "type":"LineString",
                    "coordinates":[]
                }}
                
    
    for coordinate in coordinates:
        geo_json['geometry']['coordinates'].append(list(coordinate))
    
    return geo_json

def shortest_path_to_geojson(g, start, end):
    if isinstance(start, int): 
        start = str(start)
    if isinstance(end, int):
        end = str(end)
        
    sp = nx.shortest_path(g,start,end, weight='weight')
    return coordinates_to_geojson(path_to_coordinates(g, sp))

def shortest_paths_to_geojson(g, start_end_pairs):
    '''Given a list of star/end pairs, find the shortest path and return it as a series of linestrings in geojson'''
    geo_json = {"type":"FeatureCollection",
                "features":[]
               }
    geo_json['features'] = [shortest_path_to_geojson(g, start, end) for start, end in start_end_pairs]
    return json.dumps(geo_json)
                        
        
with open('path.json', 'w') as pf:
    pf.write(shortest_paths_to_geojson(g, [(63100, 300830), (5050, 65450), (490,62870), (62570,300730), (67050, 64470)]))
#shortest_path_to_geojson(g, 63100, 300830)
#g.nodes['64470']['coordinates']
#path_distance(g,sp)

NameError: name 'g' is not defined

In [185]:
g.edges[('61230', '7_1')], 

({'weight': 10.497646606487569},)

In [186]:
g.edges[('226020', '7_1')]

{'weight': 13.906044211540713}

In [163]:
g.nodes['82_1'],g['82_1'],g.nodes['11300']

({'coordinates': (-88.58400999999998, 37.08857000000006)},
 AtlasView({'170': {'weight': 1}, '11300': {'weight': 0}}),
 {'coordinates': (-88.58400999999998, 37.08857000000006)})

In [204]:
for (u, v, wt) in g.edges.data('weight'):
    print(u,v,wt)

65550 697_1 27.275193181056004
60800 697_4 12.057993952821914
697_1 697_2 5.011015806098512
697_2 697_3 11.903326259845173
697_3 697_4 0.3155366949771066


In [207]:
sum([tup[2] for tup in g.edges.data('weight')])

408.36878829357

In [220]:
sp = nx.shortest_path(g,'63100','62290', weight='weight')

In [240]:
a = shortest_path_to_geojson(g, 490, 62870)

In [241]:
a

{'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'LineString',
  'coordinates': [[-92.70955999999995, 45.321740000000034],
   [-92.71199999999999, 45.320020000000056],
   [-92.71461999999997, 45.31832000000003],
   [-92.71710999999999, 45.316680000000076],
   [-92.72013999999996, 45.31443000000007],
   [-92.72221999999994, 45.312550000000044],
   [-92.72544999999997, 45.310820000000035],
   [-92.72830999999996, 45.309090000000026],
   [-92.72978999999998, 45.307300000000055],
   [-92.73067999999995, 45.30568000000005],
   [-92.73517999999996, 45.302130000000034],
   [-92.73713999999995, 45.30037000000004],
   [-92.73924999999997, 45.29992000000004],
   [-92.74124999999998, 45.29896000000008],
   [-92.74333999999999, 45.297570000000064],
   [-92.74517999999995, 45.297270000000026],
   [-92.74647999999996, 45.29649000000006],
   [-92.74805999999995, 45.29545000000007],
   [-92.74921999999998, 45.29427000000004],
   [-92.75019999999995, 45.29370000000006],
   [-92.751509999999